# DBScan
**D**ensity-**B**ased **S**patial **C**lustering of **A**pplications with **N**oise (dt. Dichtebasiertes räumliches Clustering von Anwendungen mit Rauschen) verwendet einen dichtebasierten Ansatz, bei dem jeder Punkt anhand seiner Umgebung bewertet wird. Zu einem Cluster gehören immer die Punkte, die untereinander "dichte-verbunden" sind. Da keine berechneten Repräsentanten für die Cluster gesucht werden, sind beliebige Formen möglich.

Im Gegensatz zu k-Means ist es bei DBScan nicht notwendig, die Anzahl der Cluster vorher festzulegen. Jedoch müssen $\epsilon$ und $minPts$ passend gewählt werden. DBScan wird außerdem weniger stark von Rauschen beeinflusst.

In [ ]:
import numpy as np

from tui_dsmt.clustering import animate_dbscan, interactive_dbscan
from tui_dsmt.clustering.datasets import clustering_example1, clustering_example2

## Inhaltsverzeichnis
- [Der Algorithmus](#Der-Algorithmus)
- [Visualisierung](#Visualisierung)
- [Erweiterte Implementierungen](#Erweiterte-Implementierungen)
- [Randnotiz](#Randnotiz)

## Der Algorithmus
DBScan benötigt zwei Parameter:
- `eps`: $\epsilon$ beschreibt einen Abstand, den zwei Punkte untereinander maximal besitzen dürfen, um noch als Nachbarn zu zählen.
- `minPts`: $minPts$ gibt an, wie viele Punkte sich mindestens in einer Nachbarschaft befinden müssen, um eine dichte Region zu bilden.

Für den Algorithmus existieren drei verschiedene Arten von Punkten:
- *Kernobjekte* besitzen in einem Umkreis mit dem Radius $\epsilon$ mindestens $minPts$ Punkte.
- *Randobjekte* sind dichte-erreichbar, d.h. sie liegen im Radius eines Kernobjekts, sind aber selbst auf Grund zu weniger Nachbarn keine Kernobjekte.
- *Rauschen* sind übrig bleibende Punkte, die zu keinem Cluster zugeordnet werden und stattdessen als Ausreißer oder Messfehler angesehen werden.

Der Algorithmus besitzt drei Funktionen:
1. `dbscan` wählt Punkte aus, die noch zu keinem Cluster gehören und nicht als Rauschen gekennzeichnet sind.

In [ ]:
def dbscan(xs, ys, eps, min_pts):
    # Zuerst wird eine Liste mit Clustern angelegt.
    clusters = [None for _ in xs]
    yield clusters

    # Der aktuelle Klassenindex wird auf 1 gesetzt.
    C = 1

    # Es wird über alle Punkte iteriert.
    for P in range(len(xs)):
        # Punkte, die bereits Teil einer Klasse oder Rauschen sind,
        # werden übersprungen.
        if clusters[P] is not None and clusters[P] > 0:
            continue

        # N ist die Menge aller Punkte in der Umgebung von P.
        # P ist ebenfalls in N enthalten!
        N = region_query(xs, ys, eps, P)

        # Falls die Umgebung zu klein ist, wird P als Rauschen
        # deklariert.
        if len(N) < min_pts:
            clusters[P] = 0
            yield clusters

        # Sonst wird der Klassenindex um 1 erhöht und die Funktion
        # expand_cluster aufgerufen, um einen Cluster um P zu
        # finden.
        else:
            yield from expand_cluster(xs, ys, eps, min_pts, clusters, N, C, P)
            C += 1

2. `expand_cluster` sucht in der Nachbarschaft eines Punktes weitere Punkte, die in den selben Cluster aufgenommen werden sollen.

In [ ]:
def expand_cluster(xs, ys, eps, min_pts, clusters, N, C, P):
    # Die Zugehörigkeit von P zum Cluster C wird gespeichert.
    clusters[P] = C
    yield clusters

    # Zur Iteration wird ein Index verwendet, da im weiteren
    # Verlauf die Menge N erweitert wird.
    Qi = 0

    while Qi < len(N):
        # Q ist der Punkt an Position Q_i in der Menge N.
        Q = N[Qi]
        Qi += 1

        # Falls Q noch nicht besucht wurde, wird die
        # Region M um Q gesucht und, falls sie selbst
        # ausreichend Elemente enthält, mit N vereinigt.
        if clusters[Q] is None:
            M = region_query(xs, ys, eps, Q)
            if len(M) >= min_pts:
                N.extend(M)

        # Q wird dem aktuellen Cluster zugeordnet.
        if clusters[Q] is None or clusters[Q] == 0:
            clusters[Q] = C
            yield clusters

3. `region_query` gibt alle Punkte im Umkreis eines Punktes zurück, die weniger als $\epsilon$ entfernt sind.

In [ ]:
def region_query(xs, ys, eps, P):
    # Die Distanzen zu allen Punkten wird berechnet.
    distances = np.sqrt((xs - xs[P]) ** 2 + (ys - ys[P]) ** 2)

    # Alle Punkte, deren Distanz kleiner als eps ist,
    # werden zurückgegeben.
    return np.where(distances < eps)[0].tolist()

## Visualisierung

Die folgende Zelle lädt den spiralförmigen Datensatz, mischt ihn zufällig und wendet DBScan Schritt für Schritt darauf an.

In [ ]:
df = clustering_example2.sample(frac=1).reset_index()
animate_dbscan(dbscan, df, eps=2, min_pts=3)

Auch auf den Datensatz, mit dem k-Means bereits gute Ergebnisse erzielte, lässt sich DBScan anwenden. Dann müssen jedoch die Werte `eps` und `min_pts` angepasst werden. Punkte, die DBScan als Rauschen erkennt, lassen sich dabei sehr gut ausmachen.

In [ ]:
df = clustering_example1.sample(frac=1).reset_index()
animate_dbscan(dbscan, df, eps=0.3, min_pts=5)

## Erweiterte Implementierungen
Auch für DBScan existiert eine Implementierung in `sklearn`. Initialisieren Sie ein Objekt der Klasse `DBSCAN` und übergeben Sie passende Werte für `eps` und `min_samples` (`min_pts`). Rufen Sie anschließend `fit_predict` mit einem DataFrame auf. Die Rückgabe enthält ein Array mit Klassenzuordnungen.

In [ ]:
from sklearn.cluster import DBSCAN

eps=2.4
min_samples=5
DBSCAN(eps=eps, min_samples=min_samples).fit_predict(df)

In der nachfolgenden Zelle haben Sie die Möglichkeit mit den Parametern $eps$ und $min\_samples$ zu experimentieren.

In [ ]:
interactive_dbscan(clustering_example1)

## Randnotiz
Auch wenn DBScan bereits mehrere Jahrzehnte alt ist, werden Abwandlungen dieses Algorithmus noch immer eingesetzt. So verwendet beispielsweise die [freie Foto-Verwaltungssoftware Immich](https://immich.app/) DBScan, um [erkannte Gesichter zu clustern](https://immich.app/docs/features/facial-recognition/) und somit Personen zuzordnen.